# import

In [1]:
import os

# os.environ["MUJOCO_GL"] = "egl"
os.environ["MUJOCO_GL"] = "osmesa"
# os.environ["MUJOCO_GL"] = "glfw"

os.environ["OPENBLAS_NUM_THREADS"] = "1"


In [2]:
from dreamer.algorithms.dreamer import Dreamer
from dreamer.algorithms.plan2explore import Plan2Explore
from dreamer.utils.utils import load_config, get_base_directory
from dreamer.envs.envs import make_dmc_env, make_atari_env, get_env_infos

# load config and init model

In [3]:
config_arg = "p2e-dmc-walker-walk"
# device = None
device ="cuda:5"

In [4]:
config = load_config(config_arg)

if device is None:
    device = config.operation.device
else:
    config["operation"]["device"] = device
print("use device: ", device)

print("local: {}, args:{}".format(device, config.operation.device))

writer = None

log_dir_base_path = os.path.join(get_base_directory(), "runs")
log_dir_name = "2023-10-27_20-13-23_batch-15-walker-walk"
log_dir = os.path.join(log_dir_base_path, log_dir_name)
print("log_dir_path: ", log_dir)

use device:  cuda:5
local: cuda:5, args:cuda:5
log_dir_path:  /root/sharespace2/plan2explore/torch-ver/SimpleDreamer/runs/2023-10-27_20-13-23_batch-15-walker-walk


In [5]:
if config.environment.benchmark == "atari":
    env = make_atari_env(
        task_name=config.environment.task_name,
        seed=config.environment.seed,
        height=config.environment.height,
        width=config.environment.width,
        skip_frame=config.environment.frame_skip,
        pixel_norm=config.environment.pixel_norm,
    )
elif config.environment.benchmark == "dmc":
    env = make_dmc_env(
        domain_name=config.environment.domain_name,
        task_name=config.environment.task_name,
        seed=config.environment.seed,
        visualize_reward=config.environment.visualize_reward,
        from_pixels=config.environment.from_pixels,
        height=config.environment.height,
        width=config.environment.width,
        frame_skip=config.environment.frame_skip,
        pixel_norm=config.environment.pixel_norm,
    )
obs_shape, discrete_action_bool, action_size = get_env_infos(env)


/root/.pyenv/versions/3.8.0/lib/python3.8/site-packages/gym/envs/registration.py:440: UserWarning: WARN: The `registry.env_specs` property along with `EnvSpecTree` is deprecated. Please use `registry` directly as a dictionary instead.
  logger.warn(
/root/.pyenv/versions/3.8.0/lib/python3.8/site-packages/dmc2gym/wrappers.py:10: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dim = np.int(np.prod(s.shape))
/root/.pyenv/versions/3.8.0/lib/python3.8/site-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which ret

In [6]:
if config.algorithm == "dreamer-v1":
    agent = Dreamer(obs_shape, discrete_action_bool, action_size, writer, device, config, log_dir)
elif config.algorithm == "plan2explore":
    agent = Plan2Explore(obs_shape, discrete_action_bool, action_size, writer, device, config, log_dir)
print("finish the initialization of agent")

model_path = agent.generate_model_path(itr=800, save_dir=log_dir)
agent.load_model(model_path)

print("Done loading the model")

finish the initialization of agent
load the model from /root/sharespace2/plan2explore/torch-ver/SimpleDreamer/runs/2023-10-27_20-13-23_batch-15-walker-walk/plan2explore_800.pth
Done loading the model


In [7]:
agent.device

'cuda:5'

# run the model in test environment

In [8]:
result_dict = agent.validation_interaction(agent.intrinsic_actor, env, 1)

  0%|          | 0/1 [00:00<?, ?it/s]/root/.pyenv/versions/3.8.0/lib/python3.8/site-packages/gym/utils/passive_env_checker.py:174: UserWarning: WARN: Future gym versions will require that `Env.reset` can be passed a `seed` instead of using `Env.seed` for resetting the environment random number generator.
  logger.warn(
/root/.pyenv/versions/3.8.0/lib/python3.8/site-packages/gym/utils/passive_env_checker.py:190: UserWarning: WARN: Future gym versions will require that `Env.reset` can be passed `return_info` to return information from the environment resetting.
  logger.warn(
/root/.pyenv/versions/3.8.0/lib/python3.8/site-packages/gym/utils/passive_env_checker.py:195: UserWarning: WARN: Future gym versions will require that `Env.reset` can be passed `options` to allow the environment initialisation to be passed additional information.
  logger.warn(
/root/.pyenv/versions/3.8.0/lib/python3.8/site-packages/gym/core.py:49: DeprecationWarning: WARN: You are calling render method, but you did

In [9]:
import numpy as np

result_dict["0"]["obs_lst"].shape
# result_dict["0"]["reward_lst"].shape

(500, 3, 64, 64)

In [10]:
np.min(result_dict["0"]["obs_lst"])

-0.46862745098039216

In [12]:
# import ffmpeg
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation

episode_key = "0"

# convert the observation
obs_lst = np.add(result_dict[episode_key]["obs_lst"].transpose(0, 2, 3, 1), 0.5)

reward_lst = result_dict[episode_key]["reward_lst"]
episode_length = obs_lst.shape[0]

fig = plt.figure(figsize=(3*1, 3*1))
ax1 = fig.add_subplot(1,1,1)

def plot(t):
    plt.cla()
    ax1.cla()
    ax1.imshow(obs_lst[t])
    ax1.set_title("time: {}, reward: {:.5f}".format(t, reward_lst[t]))
    ax1.set_axis_off()

anim = FuncAnimation(fig, plot, frames=episode_length, interval=100)
save_path = os.path.join(log_dir, "test.mp4")
print(save_path)
anim.save(save_path, writer='ffmpeg')
plt.close()

/root/sharespace2/plan2explore/torch-ver/SimpleDreamer/runs/2023-10-27_20-13-23_batch-15-walker-walk/test.mp4
